VOLVO

In [ ]:
import pandas as pd
import pdfplumber
import re
import os

arquivos = {
    "FM 4x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-4x2R.pdf",
    "FM 4x2T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-4x2T.pdf",
    "FM 6x2T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-6x2T.pdf",
    "FM 4x2R (v2)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\Ficha Técnica FM 4x2R.pdf",
    "FM 4x2T (v2)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\Ficha Técnica FM 4x2T.pdf",
    "FM 6x2T (v2)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\Ficha Técnica FM 6x2T.pdf",
    "FMX 6X6R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6R.pdf",
    "FMX 6x4T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4T.pdf",
    "FMX 6x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4R.pdf",
    "FMX 4x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-4x4R.pdf",
    "FMX 6x6T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6T.pdf",
    "FMX 8x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-8x4R.pdf",
    "FMX 6X6R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6R-Max.pdf",
    "FMX 6x4R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4R-Max.pdf",
    "FMX 8x4R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-8x4R-Max.pdf",
    "VM 4x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2R.pdf",
    "VM 4x2T":r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2T.pdf",
    "VM 6x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x2R.pdf",
    "VM 6x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x4R.pdf",
    "VM 6x4 (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x4R-Max.pdf",
    "VM 8x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x2R.pdf",
    "VM 8x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x4R.pdf",
    "VM 4x2 (CITY)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2R-City.pdf",
    "VM 6x2 (CITY)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x2R-City.pdf",
    "VM 8x4 (LIGTH MIXER)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x4R-Light-Mixer.pdf",
    "FH 6x4T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\fh6x4-T.pdf",
}

def ajustar_tamanho(lista, tamanho):
    return lista[:tamanho] + ["N/A"] * (tamanho - len(lista))

def extrair_dimensoes_pesos(texto, modelo_nome):
    versoes = re.findall(r"VERSÃO\s+((?:\d{4}\s+)+)", texto)
    versoes = versoes[0].split() if versoes else []
    n = len(versoes) if versoes else 1

    def extrair_valores(label):
        padrao = rf"{label}[^\d]*(\d[\d\/\s]*)"
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            valores = re.findall(r"\d+/?\d*", match.group(1))
            return ajustar_tamanho(valores, n)
        return ["N/A"] * n

    return pd.DataFrame({
        "Modelo": [modelo_nome] * n,
        "Versão": versoes if versoes else ["N/A"] * n,
        "Entre eixos (mm)": extrair_valores("Entre eixos"),
        "Balanço dianteiro (mm)": extrair_valores("Balanço dianteiro"),
        "Dist. eixo dianteiro/implemento (mm)": extrair_valores("Distância eixo dianteiro/implemento"),
        "Balanço traseiro (mm)": extrair_valores("Balanço traseiro"),
        "Comprimento total (mm)": extrair_valores("Comprimento total"),
        "Altura cabine (mm)": extrair_valores("Altura da cabine"),
        "Largura cabine (mm)": extrair_valores("Largura da cabine"),
        "Raio de giro (mm)": extrair_valores("Raio de giro"),
        "Tara dianteira (kg)": extrair_valores("Tara eixo dianteiro"),
        "Tara traseira (kg)": extrair_valores("Tara no eixo traseiro"),
        "Tara total (kg)": extrair_valores("Tara total do chassi"),
        "Tanque combustível (L)": extrair_valores("Tanque de combustível"),
    })

# Inicializa as listas
lista_dimensoes = []
lista_motor = []
lista_transmissao = []
modelos = []

for modelo, caminho in arquivos.items():
    if not os.path.exists(caminho):
        print(f"Aviso: Arquivo não encontrado para {modelo}: {caminho}")
        continue

    modelos.append(modelo)
    with pdfplumber.open(caminho) as pdf:
        texto = "\n".join([p.extract_text() for p in pdf.pages])
        texto_limpo = texto.replace('\n', ' ').replace('–', '-').replace('—', '-')

    # Extração: Dimensões e Pesos
    lista_dimensoes.append(extrair_dimensoes_pesos(texto, modelo))

    # Extração: Motor
    motor = re.search(r"MOTOR\s+([A-Z\d\-]+)", texto_limpo, re.IGNORECASE)
    pot = re.search(r"(\d{3,4})\s*cv\s*-\s*(\d{2,4})\s*kW", texto_limpo)
    rpm_pot = re.search(r"\((\d{4})-(\d{4})\s*rpm\)", texto_limpo)
    torque = re.search(r"(\d{4})\s*Nm", texto_limpo)
    faixa_torque = "N/A"
    if torque:
        idx = texto_limpo.find(torque.group(0))
        rpm_torque = re.search(r"\((\d{3,4})-(\d{4})\s*rpm\)", texto_limpo[idx:])
        if rpm_torque:
            faixa_torque = f"{rpm_torque.group(1)}–{rpm_torque.group(2)}"

    lista_motor.append(pd.DataFrame([{
        "Modelo": modelo,
        "Motor": motor.group(1) if motor else "N/A",
        "Potência (cv/kW)": f"{pot.group(1)}/{pot.group(2)}" if pot else "N/A",
        "Faixa potência (rpm)": f"{rpm_pot.group(1)}–{rpm_pot.group(2)}" if rpm_pot else "N/A",
        "Torque (Nm)": torque.group(1) if torque else "N/A",
        "Faixa torque (rpm)": faixa_torque,
        "Nº cilindros": 6,
        "Válvulas por cilindro": 4,
        "Cilindrada (L)": 12.8,
        "Freio motor": "VEB+",
        "Norma de emissão": "Proconve P8",
        "Tipo de injeção": "Common Rail"
    }]))

    # Extração: Transmissão
    trans_secao = re.search(r"TRANSMISSÃO\s*(.*?)Capacidade de óleo", texto_limpo, re.DOTALL | re.IGNORECASE)
    trans_texto = trans_secao.group(1).strip() if trans_secao else ""

    # Nome da transmissão (linha logo após "TRANSMISSÃO")
    trans_nome_match = re.search(r"TRANSMISSÃO\s*([\w\- ]+)", texto, re.IGNORECASE)
    trans_nome = trans_nome_match.group(1).strip() if trans_nome_match else "N/A"

    # Tipo
    tipo_match = re.search(r"Tipo\s*:?[\s]*(Automatizada|Manual)", trans_texto, re.IGNORECASE)
    tipo = tipo_match.group(1) if tipo_match else "N/A"

    # Trocas
    modos_match = re.search(r"Trocas de marcha\s*:?[\s]*(Manual ou Autom[aá]tico)", trans_texto, re.IGNORECASE)
    modos = modos_match.group(1) if modos_match else "N/A"

    # Marchas
    marchas_f = re.search(r"marchas a frente\s*:?[\s]*(\d+)", trans_texto, re.IGNORECASE)
    marchas_r = re.search(r"marchas a r[eé]\s*:?[\s]*(\d+)", trans_texto, re.IGNORECASE)

    # Óleo
    oleo_match = re.search(r"Capacidade de óleo\s*\(?(l|L)?\)?\s*:?[\s]*(\d+)", texto_limpo, re.IGNORECASE)
    cap_oleo = oleo_match.group(2) if oleo_match else "N/A"

    # Adiciona ao DataFrame
    lista_transmissao.append(pd.DataFrame([{
        "Modelo": modelo,
        "Transmissão": trans_nome,
        "Tipo": tipo,
        "Modos": modos,
        "Marchas à frente": marchas_f.group(1) if marchas_f else "N/A",
        "Marchas à ré": marchas_r.group(1) if marchas_r else "N/A",
        "Capacidade óleo (L)": cap_oleo
    }]))

# Componentes padronizados
df_componentes = pd.DataFrame({
    "Modelo": modelos,
    "Freios - Tipo": ["Tambor/Disco com ABS e EBS"] * len(modelos),
    "Freios - Freio Motor": ["VEB+"] * len(modelos),
    "Freios - Estacionamento": ["Mola acumuladora com acionamento pneumático"] * len(modelos),
    "Eixo Dianteiro - Tipo": ['Perfil "I" em Aço Forjado'] * len(modelos),
    "Eixo Dianteiro - Cubos": ["Lubrificados a graxa e livres de manutenção"] * len(modelos),
    "Eixo Dianteiro - Carga (t)": ["7,5"] * len(modelos),
    "Embreagem - Tipo": ["Monodisco de fricção a seco tipo push-type"] * len(modelos),
    "Embreagem - Acionamento": ["Automático (I-Shift)"] * len(modelos),
    "Embreagem - Disco (mm)": ["430"] * len(modelos),
    "Suspensão Dianteira - Tipo": ["Parabólica"] * len(modelos),
    "Suspensão Dianteira - Amortecedores": ["2 de dupla ação"] * len(modelos),
    "Suspensão Dianteira - Barra estabilizadora": ["Sim"] * len(modelos),
    "Eixos Traseiros - Tipo": ["Simples velocidade"] * len(modelos),
    "Eixos Traseiros - Redução nos cubos": ["Não / Sim"] * len(modelos),
    "Eixos Traseiros - Tipo de carcaça": ["Fundida"] * len(modelos),
    "Eixos Traseiros - CMT (t)": ["60 / 70"] * len(modelos),
    "Eixos Traseiros - Relações de redução": [", ".join([
        "2.85:1", "3.08:1", "3.40:1", "3.67:1", "3.46:1", 
        "3.61:1", "3.76:1", "4.12:1", "4.55:1", "5.41:1"
    ])] * len(modelos),
    "Eixos Traseiros - Capacidade óleo (l)": ["14,5 / 23"] * len(modelos),
    "Eixos Traseiros - Bloqueio diferencial": ["Sim / Sim"] * len(modelos),
    "Chassi - Tipo": ['Perfil "U"'] * len(modelos),
    "Chassi - Material": ["LNE60"] * len(modelos),
    "Chassi - Bitola (mm)": ["850"] * len(modelos),
    "Chassi - Altura da alma (mm)": ["300"] * len(modelos),
    "Chassi - Largura da aba (mm)": ["90"] * len(modelos),
    "Chassi - Espessura longarina (mm)": ["8"] * len(modelos),
    "Chassi - Tipo de reforço": ["5"] * len(modelos),
    "Elétrico - Tensão nominal": ["24V"] * len(modelos),
    "Elétrico - Bateria": ["2x170Ah / 12V ou 2x225Ah / 12V"] * len(modelos),
    "Elétrico - Alternador": ["110A/28V ou 120A/28V ou 150A/28V"] * len(modelos),
    "Cabine - Tipo": ["CAB-DAY / CAB-SLP / CAB-HSLP"] * len(modelos),
    "Cabine - Tipo de suspensão": ["Mola ou Pneumática"] * len(modelos)
})

# Exporta para Excel
saida = r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Volvo Base.xlsx"
with pd.ExcelWriter(saida) as writer:
    pd.concat(lista_dimensoes).to_excel(writer, sheet_name="Dimensoes e Pesos", index=False)
    pd.concat(lista_motor).to_excel(writer, sheet_name="Motor", index=False)
    pd.concat(lista_transmissao).to_excel(writer, sheet_name="Transmissao", index=False)
    df_componentes.to_excel(writer, sheet_name="Componentes Expandidos", index=False)

print(f"Excel final salvo com sucesso: {saida}")

In [ ]:
import pandas as pd
import pdfplumber
import re
import os

arquivos = {
    "FM 4x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-4x2R.pdf",
    "FM 4x2T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-4x2T.pdf",
    "FM 6x2T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FM-6x2T.pdf",
    "FMX 6X6R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6R.pdf",
    "FMX 6x4T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4T.pdf",
    "FMX 6x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4R.pdf",
    "FMX 4x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-4x4R.pdf",
    "FMX 6x6T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6T.pdf",
    "FMX 8x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-8x4R.pdf",
    "FMX 6X6R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x6R-Max.pdf",
    "FMX 6x4R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-6x4R-Max.pdf",
    "FMX 8x4R (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\FMX-8x4R-Max.pdf",
    "VM 4x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2R.pdf",
    "VM 4x2T":r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2T.pdf",
    "VM 6x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x2R.pdf",
    "VM 6x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x4R.pdf",
    "VM 6x4 (MAX)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x4R-Max.pdf",
    "VM 8x2R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x2R.pdf",
    "VM 8x4R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x4R.pdf",
    "VM 4x2 (CITY)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-4x2R-City.pdf",
    "VM 6x2 (CITY)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-6x2R-City.pdf",
    "VM 8x4 (LIGTH MIXER)": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\VM-8x4R-Light-Mixer.pdf",
    "FH 6x4T": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas VOLVO\fh6x4-T.pdf",
}

def ajustar_tamanho(lista, tamanho):
    return lista[:tamanho] + ["N/A"] * (tamanho - len(lista))

def extrair_dimensoes_pesos(texto, modelo_nome):
    versoes = re.findall(r"VERSÃO\s+((?:\d{4}\s+)+)", texto)
    versoes = versoes[0].split() if versoes else []
    n = len(versoes) if versoes else 1

    def extrair_valores(label):
        padrao = rf"{label}[^\d]*((?:[\d\+\*/\s\(\)]+[/\*]?[\s\w]*)+)"
        match = re.search(padrao, texto, re.IGNORECASE)
        if match:
            valores_brutos = match.group(1).strip()
            valores = re.findall(r"\d+(?:\s*[\+*/]\s*\d+)*(?:\s*\([^)]+\))?", valores_brutos)
            return ajustar_tamanho(valores, n)
        return ["N/A"] * n

    return pd.DataFrame({
        "Modelo": [modelo_nome] * n,
        "Versão": versoes if versoes else ["N/A"] * n,
        "Entre eixos (mm)": extrair_valores("Entre eixos"),
        "Balanço dianteiro (mm)": extrair_valores("Balanço dianteiro"),
        "Dist. eixo dianteiro/implemento (mm)": extrair_valores("Distância eixo dianteiro/implemento"),
        "Balanço traseiro (mm)": extrair_valores("Balanço traseiro"),
        "Comprimento total (mm)": extrair_valores("Comprimento total"),
        "Altura cabine (mm)": extrair_valores("Altura da cabine"),
        "Largura cabine (mm)": extrair_valores("Largura da cabine"),
        "Raio de giro (mm)": extrair_valores("Raio de giro"),
        "Tara dianteira (kg)": extrair_valores("Tara eixo dianteiro"),
        "Tara traseira (kg)": extrair_valores("Tara no eixo traseiro"),
        "Tara total (kg)": extrair_valores("Tara total do chassi"),
        "Tanque combustível (L)": extrair_valores("Tanque de combustível"),
    })

def extrair_motor(texto, modelo):
    texto = texto.replace('\n', ' ').replace('–', '-').replace('—', '-').replace('\u2010', '-')

    # Motor
    motor_match = re.search(r"(D\d{1,2}[KC]\s*[-–—]?\s*\d{3}|D\d{1,2}[KC]\d{3})", texto, re.IGNORECASE)
    motor = motor_match.group(0).replace(" ", "").replace("–", "-").replace("—", "-") if motor_match else "N/A"

    # Potência (cv/kW) - padrão 1
    pot_match = re.search(r"(\d{2,4})\s*cv\s*[-–]?\s*(\d{2,4})\s*kW", texto)
    if pot_match:
        pot_cv = pot_match.group(1)
        pot_kw = pot_match.group(2)
    else:
        # fallback: só cv com rpm
        pot_match_alt = re.search(r"(\d{2,4})\s*cv[^@\d]*@\s*(\d{4})\s*rpm", texto)
        pot_cv = pot_match_alt.group(1) if pot_match_alt else "N/A"
        pot_kw = "N/A"

    # Faixa de potência
    faixa_pot = "N/A"
    faixa_rpm = re.search(r"\((\d{4})\s*[-–—]\s*(\d{4})\s*rpm\)", texto)
    if faixa_rpm:
        faixa_pot = f"{faixa_rpm.group(1)}–{faixa_rpm.group(2)}"
    else:
        pot_rpm = re.search(r"@\s*(\d{4})\s*rpm", texto)
        if pot_rpm:
            faixa_pot = f"{pot_rpm.group(1)} rpm"

    # Torque
    torque_match = re.search(r"(\d{3,4})\s*Nm", texto)
    torque = torque_match.group(1) if torque_match else "N/A"

    # Faixa torque (rpm)
        # Faixa torque (rpm)
        # Faixa torque (rpm)
    faixa_torque = "N/A"
    match_1 = re.search(r"\d{3,4}\s*Nm\s*@\s*(\d{4})\s*[-–—]\s*(\d{4})\s*rpm", texto)
    match_2 = re.search(r"(?:faixa\s+(?:de\s+torque|de\s+rotação)[^\d]{0,10})?(\d{4})\s*[-–—]\s*(\d{4})\s*rpm", texto, re.IGNORECASE)
    if match_1:
        faixa_torque = f"{match_1.group(1)}–{match_1.group(2)}"
    elif match_2:
        faixa_torque = f"{match_2.group(1)}–{match_2.group(2)}"


    cilindrada = 7.7 if "D8K" in motor else 12.8

    return pd.DataFrame([{
        "Modelo": modelo,
        "Motor": motor,
        "Potência (cv/kW)": f"{pot_cv}/{pot_kw}" if pot_cv != "N/A" else "N/A",
        "Faixa potência (rpm)": faixa_pot,
        "Torque (Nm)": torque,
        "Faixa torque (rpm)": faixa_torque,
        "Nº cilindros": 6,
        "Válvulas por cilindro": 4,
        "Cilindrada (L)": cilindrada,
        "Freio motor": "VEB+",
        "Norma de emissão": "Proconve P8",
        "Tipo de injeção": "Common Rail"
    }])

def extrair_transmissao(texto, modelo):
    texto = texto.replace('\n', ' ').replace('–', '-').replace('—', '-')

    # Nome da transmissão
    trans_nome = re.search(r"(I-Shift\s+[A-Z0-9\-]+|Caixa\s+Manual\s+[A-Z0-9\-]+|ZT\d{4,5})", texto, re.IGNORECASE)
    if not trans_nome:
        trans_nome = re.search(r"Transmiss[aã]o\s*[:\-]?\s*(\S+)", texto, re.IGNORECASE)

    # Tipo
    tipo = re.search(r"(Automatizada|Manual)", texto, re.IGNORECASE)

    # Modos
    modos = re.search(r"Trocas de marcha\s*[:\-]?\s*(Manual ou Autom[áa]tico)", texto, re.IGNORECASE)
    if not modos:
        modos = re.search(r"(Sincronizada)", texto, re.IGNORECASE)

    # Marchas
    marchas_f = re.search(r"marchas a frente\s*[:\-]?\s*(\d+)", texto, re.IGNORECASE)
    marchas_r = re.search(r"marchas a r[ée]\s*[:\-]?\s*(\d+)", texto, re.IGNORECASE)

    # Óleo
    oleo = re.search(r"Capacidade de óleo\s*\(?(l|L)?\)?\s*[:\-]?\s*(\d{1,3}(?:,\d+)?)", texto, re.IGNORECASE)
    if not oleo:
        oleo = re.search(r"óleo\s*[:\-]?\s*(\d{1,3}(?:,\d+)?)\s*l", texto, re.IGNORECASE)

    return pd.DataFrame([{
        "Modelo": modelo,
        "Transmissão": trans_nome.group(0).strip() if trans_nome else "N/A",
        "Tipo": tipo.group(1) if tipo else "N/A",
        "Modos": modos.group(1) if modos else "N/A",
        "Marchas à frente": marchas_f.group(1) if marchas_f else "N/A",
        "Marchas à ré": marchas_r.group(1) if marchas_r else "N/A",
        "Capacidade óleo (L)": oleo.group(2) if oleo else "N/A"
    }])

# Coleta de dados
lista_dimensoes = []
lista_motor = []
lista_transmissao = []
modelos = []

for modelo, caminho in arquivos.items():
    if not os.path.exists(caminho):
        print(f"Aviso: Arquivo não encontrado para {modelo}: {caminho}")
        continue

    modelos.append(modelo)
    with pdfplumber.open(caminho) as pdf:
        texto = "\n".join([p.extract_text() for p in pdf.pages])
    
    lista_dimensoes.append(extrair_dimensoes_pesos(texto, modelo))
    lista_motor.append(extrair_motor(texto, modelo))
    lista_transmissao.append(extrair_transmissao(texto, modelo))

modelos_final = modelos
n = len(modelos_final)
df_componentes = pd.DataFrame({
    "Modelo": modelos_final,
    "Freios - Tipo": ["Tambor/Disco com ABS e EBS"] * n,
    "Freios - Freio Motor": ["VEB+"] * n,
    "Freios - Estacionamento": ["Mola acumuladora com acionamento pneumático"] * n,
    "Eixo Dianteiro - Tipo": ['Perfil "I" em Aço Forjado'] * n,
    "Eixo Dianteiro - Cubos": ["Lubrificados a graxa e livres de manutenção"] * n,
    "Eixo Dianteiro - Carga (t)": ["7,5"] * n,
    "Embreagem - Tipo": ["Monodisco de fricção a seco tipo push-type"] * n,
    "Embreagem - Acionamento": ["Automático (I-Shift)"] * n,
    "Embreagem - Disco (mm)": ["430"] * n,
    "Suspensão Dianteira - Tipo": ["Parabólica"] * n,
    "Suspensão Dianteira - Amortecedores": ["2 de dupla ação"] * n,
    "Suspensão Dianteira - Barra estabilizadora": ["Sim"] * n,
    "Eixos Traseiros - Tipo": ["Simples velocidade"] * n,
    "Eixos Traseiros - Redução nos cubos": ["Não / Sim"] * n,
    "Eixos Traseiros - Tipo de carcaça": ["Fundida"] * n,
    "Eixos Traseiros - CMT (t)": ["60 / 70"] * n,
    "Eixos Traseiros - Relações de redução": [", ".join(["2.85:1", "3.08:1", "3.40:1", "3.67:1", "3.46:1", "3.61:1", "3.76:1", "4.12:1", "4.55:1", "5.41:1"])] * n,
    "Eixos Traseiros - Capacidade óleo (l)": ["14,5 / 23"] * n,
    "Eixos Traseiros - Bloqueio diferencial": ["Sim / Sim"] * n,
    "Chassi - Tipo": ['Perfil "U"'] * n,
    "Chassi - Material": ["LNE60"] * n,
    "Chassi - Bitola (mm)": ["850"] * n,
    "Chassi - Altura da alma (mm)": ["300"] * n,
    "Chassi - Largura da aba (mm)": ["90"] * n,
    "Chassi - Espessura longarina (mm)": ["8"] * n,
    "Chassi - Tipo de reforço": ["5"] * n,
    "Elétrico - Tensão nominal": ["24V"] * n,
    "Elétrico - Bateria": ["2x170Ah / 12V ou 2x225Ah / 12V"] * n,
    "Elétrico - Alternador": ["110A/28V ou 120A/28V ou 150A/28V"] * n,
    "Cabine - Tipo": ["CAB-DAY / CAB-SLP / CAB-HSLP"] * n,
    "Cabine - Tipo de suspensão": ["Mola ou Pneumática"] * n,
})

for modelo, caminho in arquivos.items():
    if not os.path.exists(caminho):
        print(f"Arquivo não encontrado: {modelo}")
        continue
    with pdfplumber.open(caminho) as pdf:
        texto = "\n".join([p.extract_text() for p in pdf.pages])

    lista_motor.append(extrair_motor(texto, modelo))
    lista_transmissao.append(extrair_transmissao(texto, modelo))

# Salvar
saida = r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Volvo Base.xlsx"
with pd.ExcelWriter(saida) as writer:
    pd.concat(lista_dimensoes).to_excel(writer, sheet_name="Dimensoes e Pesos", index=False)
    pd.concat(lista_motor).to_excel(writer, sheet_name="Motor", index=False)
    pd.concat(lista_transmissao).to_excel(writer, sheet_name="Transmissao", index=False)
    df_componentes.to_excel(writer, sheet_name="Componentes Expandidos", index=False)

print(f"Excel final salvo com sucesso: {saida}")

Limpando Base

In [ ]:
import pandas as pd

arquivo_original = r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Volvo Base.xlsx"

arquivo_limpo = arquivo_original.replace(".xlsx", " - Limpo.xlsx")

abas = pd.read_excel(arquivo_original, sheet_name=None)

abas_limpas = {}

for nome_aba, df in abas.items():
    print(f"Limpando duplicatas da aba: {nome_aba}")
    df_limpo = df.drop_duplicates(keep='first').reset_index(drop=True)
    abas_limpas[nome_aba] = df_limpo

with pd.ExcelWriter(arquivo_limpo, engine='openpyxl') as writer:
    for nome_aba, df in abas_limpas.items():
        df.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo limpo salvo em: {arquivo_limpo}")

FORD

In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import re
import pandas as pd
import os
from pathlib import Path
import logging

# Configurar o caminho para o Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Dicionário com os arquivos PDF
arquivos_ford = {
    "C-816": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-816-especificacoes-tecnicas.pdf",
    "C-1119": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1119-especificacoes-tecnicas.pdf",
    "C-1419": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1419-especificacoes-tecnicas.pdf",
    "C-1519": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1519-especificacoes-tecnicas.pdf",
    "C-1719": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1719-especificacoes-tecnicas.pdf",
    "C-1723": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1723-especificacoes-tecnicas.pdf",
    "C-1723 Kolector": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1723kolector-especificacoes-tecnicas.pdf",
    "C-1731R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1731r-especificacoes-tecnicas.pdf",
    "C-1731 Tractor": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1731-tractor-especificacoes-tecnicas.pdf",
    "C-1933R": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-1933r-especificacoes-tecnicas.pdf",
    "C-2042": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-2042-tractor-especificacoes-tecnicas.pdf",
    "C-2423": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-2423-6x2especificacoes-tecnicas.pdf",
    "C2842": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-2842-6x2-tractor-especificacoes-tecnicas.pdf",
    "C-3031": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-3031-8x2-especificacoes-tecnicas.pdf",
    "C-3131": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-3131-6x4-especificacoes-tecnicas.pdf",
    "C-3133": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\c-3133-6x4-especificacoes-tecnicas.pdf",
    "F-350": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\f-350-especificacoes-tecnicas.pdf",
    "F-4000": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\f-4000-especificacoes-tecnicas.pdf",
    "F-4000 4x4": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Fichas Tecnicas FORD\f-40004x4-especificacoes-tecnicas.pdf"
}

# Caminho de saída
saida = r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Ford Base.xlsx"

# Criar pasta para logs
log_dir = "pdf_logs"
os.makedirs(log_dir, exist_ok=True)

# Função para extrair texto de PDFs baseados em texto
def extract_text_pdfplumber(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        logger.info(f"Texto extraído com pdfplumber de {pdf_path}")
    except Exception as e:
        logger.error(f"Erro ao extrair texto com pdfplumber em {pdf_path}: {e}")
    return text

# Função para extrair texto de PDFs baseados em imagens (usando OCR)
def extract_text_ocr(pdf_path):
    text = ""
    try:
        images = convert_from_path(pdf_path)
        for i, img in enumerate(images):
            page_text = pytesseract.image_to_string(img, lang='por')
            text += page_text + "\n"
            logger.info(f"Página {i+1} extraída com OCR de {pdf_path}")
    except Exception as e:
        logger.error(f"Erro ao extrair texto com OCR em {pdf_path}: {e}")
    return text

# Função para extrair tabelas com pdfplumber
def extract_tables_pdfplumber(pdf_path):
    tables = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_tables = page.extract_tables()
                for table in page_tables:
                    tables.append(table)
        logger.info(f"Tabelas extraídas de {pdf_path}")
    except Exception as e:
        logger.error(f"Erro ao extrair tabelas de {pdf_path}: {e}")
    return tables

# Função para salvar texto extraído em arquivo de log
def save_extracted_text(modelo, text):
    log_file = os.path.join(log_dir, f"{modelo}_extracted.txt")
    with open(log_file, 'w', encoding='utf-8') as f:
        f.write(text)
    logger.info(f"Texto extraído salvo em {log_file}")

# Função para limpar valores extraídos
def clean_value(value):
    if not value:
        return None
    # Remover espaços extras e normalizar
    value = re.sub(r'\s+', ' ', value).strip()
    # Remover texto irrelevante que pode ter sido capturado
    value = re.sub(r'(bateria\s*\(quantidade/v/cca/ah\).*|óleo\s*do\s*motor.*|rodas\s*e\s*pneus.*|pneus\s*\d+.*|b\s*-\s*balanço.*|c\s*-\s*comprimento.*|\(\*\)\s*norma\s*de\s*ensaio.*)', '', value, flags=re.IGNORECASE)
    return value.strip() if value else None

# Função para converter dimensões de mm para m
def convert_mm_to_m(value):
    if not value:
        return None
    try:
        # Se o valor contém várias opções (ex.: 6140/7010/7410)
        if '/' in value:
            values = [float(v.replace('.', '').replace(',', '.')) / 1000 for v in value.split('/')]
            return '/'.join(str(v) for v in values)
        return str(float(value.replace('.', '').replace(',', '.')) / 1000)
    except (ValueError, TypeError):
        return value

# Função para extrair especificações técnicas com regex ajustadas
def extract_specifications(text):
    specs = {
        "modelo": None,
        "massa_kg": None,
        "largura_m": None,
        "altura_m": None,
        "comprimento_m": None,
        "esc": None,
        "frenagem_emergencia": None,
        "motor": {},
        "transmissao": {},
        "embreagem": {},
        "suspensao": {"dianteira": {}, "traseira": {}},
        "sistema_eletrico": {},
        "direcao": {},
        "pesos": {},
        "chassi_dimensoes": {},
        "volumes_abastecimento": {},
        "rodas_e_pneus": {},
        "freios": {}
    }

    # Normalizar o texto para lidar com quebras de linha, espaços e caracteres especiais
    text = text.encode('utf-8', 'ignore').decode('utf-8')
    text = re.sub(r'\s+', ' ', text).lower()

    # Patterns para as especificações gerais
    patterns = {
        "massa_kg": r"(?:peso\s*bruto\s*total\s*\(pbt\)\s*homologado)\s*[:\s]*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s*(?:kg)?",
        "largura_m": r"(?:[jk]\s*-\s*largura\s*(?:externa\s*do\s*chassi|máxima\s*do\s*rodado\s*traseiro))\s*[:\s]*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s*(?:mm)?",
        "altura_m_cabina": r"(?:[gh]\s*-\s*altura\s*(?:da\s*(?:cabina|aba\s*superior\s*da\s*longarina)\s*(?:ao\s*solo|à\s*cabine)))\s*[:\s]*(1\d{3}(?:\.\d{3})*(?:,\d+)?)\s*(?:mm)?",
        "altura_m_fallback": r"(?:[gh]\s*-\s*altura\s*(?:da\s*aba\s*superior\s*da\s*longarina\s*ao\s*solo))\s*[:\s]*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s*(?:mm)?",
        "comprimento_m": r"(?:[c]\s*-\s*comprimento\s*total)\s*[:\s]*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)\s*(?:mm)?",
        "comprimento_m_fallback": r"(?:[en]\s*-\s*distância\s*entre\s*eixos)\s*[:\s]*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)\s*(?:mm)?",
        "esc": r"(?:esc|controle\s*(?:eletrônico\s*de)?\s*estabilidade|asr)(?:\s*/?\s*(?:abs|ebd))?\s*(?:[:=]|\s)*(?:presente|sim|equipado|incluído|obrigatório|disponível)?",
        "frenagem_emergencia": r"(?:frenagem\s*(?:autônoma|de\s*emergência)|aeb)\s*(?:[:=]|\s)*(?:presente|sim|equipado|incluído|obrigatório|disponível)?"
    }

    # Patterns para motor
    motor_patterns = {
        "marca_modelo": r"marca\s*/\s*modelo\s*(.+?)(?=tipo|pot[êe]ncia|torque|cilindrada|relação|sistema|$)",
        "tipo": r"tipo\s*([^\n]+?)(?=pot[êe]ncia|torque|cilindrada|relação|sistema|$)",
        "potencia_maxima": r"pot[êe]ncia\s*m[áa]xima\s*[-:]?\s*(\d+cv\s*\(\d+kW\)\s*a\s*\d+\s*rpm)",
        "torque_maximo": r"torque\s*m[áa]ximo\s*[-:]?\s*([\d,]+\s*kgfm\s*\(\d+\s*Nm\)\s*/\s*\d+(?:\s*-\s*\d+)?\s*rpm)",
        "cilindrada_total": r"cilindrada\s*total\s*\(cm3\)\s*(\d{3,5})",
        "relacao_compressao": r"rela[cç][aã]o\s*de\s*compress[aã]o\s*(\d+,\d+\s*:\s*\d+)",
        "sistema_injecao": r"sistema\s*de\s*inje[cç][aã]o\s*([^\n]+)"
    }

    # Patterns para transmissão
    transmissao_patterns = {
        "marca_modelo": r"marca\s*/\s*modelo\s*([^\n]+)",
        "acionamento": r"acionamento\s*([^\n]+)",
        "marchas": r"marchas\s*([^\n]+)",
        "relacao_transmissao_1a": r"1[ªa] marcha:\s*([\d,]+:\d+)",
        "relacao_transmissao_2a": r"2[ªa] marcha:\s*([\d,]+:\d+)",
        "relacao_transmissao_3a": r"3[ªa] marcha:\s*([\d,]+:\d+)",
        "relacao_transmissao_4a": r"4[ªa] marcha:\s*([\d,]+:\d+)",
        "relacao_transmissao_5a": r"5[ªa] marcha:\s*([\d,]+:\d+)",
        "relacao_transmissao_re": r"ré:\s*([\d,]+:\d+)"
    }

    # Patterns para embreagem
    embreagem_patterns = {
        "marca": r"embreagem\s*marca\s*(\w+)",
        "acionamento": r"acionamento\s*(hidráulico|pneumático|manual|mecânico)",
        "tipo": r"tipo\s*([^\n,]+)",
        "diametro": r"diâmetro\s*\(mm\)\s*(\d+)"
    }

    # Patterns para suspensão
    suspensao_dianteira_patterns = {
        "eixo": r"suspensão\s*dianteira\s*eixo\s*([^\s].*?)(?=\s*molas|\s*amortecedores|\s*$)",
        "molas": r"suspensão\s*dianteira\s*molas\s*([^\s].*?)(?=\s*amortecedores|\s*$)",
        "amortecedores": r"suspensão\s*dianteira\s*amortecedores\s*([^\s].*?)"
    }
    suspensao_traseira_patterns = {
        "eixo": r"suspensão\s*traseira\s*eixo\s*([^\s].*?)(?=\s*molas|\s*amortecedores|\s*$)",
        "molas": r"suspensão\s*traseira\s*molas\s*([^\s].*?)(?=\s*amortecedores|\s*$)",
        "amortecedores": r"suspensão\s*traseira\s*amortecedores\s*([^\s].*?)"
    }

    # Patterns para sistema elétrico
    sistema_eletrico_patterns = {
        "sistema": r"sistema\s*(\d+v)",
        "alternador": r"alternador\s*\(v/a\)\s*([\d\s/]+)",
        "bateria": r"bateria\s*\(quantidade/v/cca/ah\)\s*([\d\s/]+)"
    }

    # Patterns para direção
    direcao_patterns = {
        "marca_modelo": r"direção\s*marca/modelo\s*([^\n]+)",
        "tipo": r"tipo\s*([^\n]+)"
    }

    # Patterns para pesos
    pesos_patterns = {
        "pbt_homologado": r"peso\s*bruto\s*total\s*\(pbt\)\s*homologado\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s*kg",
        "carga_util": r"carga\s*útil\s*\+\s*carroceria\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)\s*kg",
        "capacidade_tracao": r"capacidade\s*máxima\s*de\s*tração\s*\(cmt\)\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s*kg"
    }

    # Patterns para chassi/dimensões
    chassi_dimensoes_patterns = {
        "entre_eixos": r"entre-eixos\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)",
        "balanco_dianteiro": r"[a]\s*-\s*balanço\s*dianteiro\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "balanco_traseiro": r"[b]\s*-\s*balanço\s*traseiro\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)",
        "comprimento_total": r"[c]\s*-\s*comprimento\s*total\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)",
        "plataforma_carga": r"[f]\s*-\s*plataforma\s*de\s*carga\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?(?:/\d{1,3}(?:\.\d{3})*(?:,\d+)?)*)",
        "altura_longarina_solo": r"[g]\s*-\s*altura\s*da\s*aba\s*superior\s*da\s*longarina\s*ao\s*solo\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "altura_longarina_cabine": r"[h]\s*-\s*altura\s*da\s*aba\s*superior\s*da\s*longarina\s*à\s*cabine\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "largura_chassi": r"[j]\s*-\s*largura\s*externa\s*do\s*chassi\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "largura_rodado_traseiro": r"[k]\s*-\s*largura\s*máxima\s*do\s*rodado\s*traseiro\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "bitola_dianteira": r"[l]\s*-\s*bitola\s*dianteira\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "bitola_traseira": r"[m]\s*-\s*bitola\s*traseira\s*(\d{1,3}(?:\.\d{3})*(?:,\d+)?)",
        "diametro_giro": r"[n]\s*-\s*diâmetro\s*de\s*giro\s*\(m\)\s*(\d+\,?\d*(?:/\d+\,?\d*)*)"
    }


    # Patterns para volumes de abastecimento
    volumes_abastecimento_patterns = {
        "tanque_combustivel": r"tanque\s*de\s*combustível\s*(\d+)",
        "reservatorio_arla": r"reservatório\s*de\s*arla\s*32\s*(\d+)",
        "oleo_motor": r"óleo\s*do\s*motor\s*(?:com\s*filtro)?\s*(\d+\.?\d*)"
    }

    # Patterns para rodas e pneus
    rodas_pneus_patterns = {
        "rodas": r"rodas\s*([^\n]+)",
        "medidas": r"medidas\s*\(em\s*polegadas\)\s*([^\n]+)",
        "pneus": r"pneus\s*([^\n]+)"
    }

    # Patterns para freios
    freios_patterns = {
        "de_servico": r"de\s*serviço\s*([\s\S]+?)(?=de\s*estacionamento|freio-motor|$)",
        "de_estacionamento": r"de\s*estacionamento\s*([^\n]+)",
        "freio_motor": r"freio-motor\s*([^\n]+)",
        "area_efetiva": r"área\s*efetiva\s*de\s*frenagem\s*\(cm²\)\s*(\d+)"
    }

    # Extrair especificações gerais
    for key, pattern in patterns.items():
        if key in specs and specs[key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            if key in ["esc", "frenagem_emergencia"]:
                if key == "esc" and "asr" in match.group(0).lower():
                    specs[key] = "Possível (ASR)"
                    logger.info(f"Encontrado {key}: Possível (ASR)")
                else:
                    specs[key] = "Sim"
                    logger.info(f"Encontrado {key}: Sim")
            else:
                value = match.group(1)
                if key in ["largura_m", "altura_m_cabina", "altura_m_fallback", "comprimento_m", "comprimento_m_fallback"]:
                    value = convert_mm_to_m(value)
                if key == "comprimento_m_fallback" and not specs["comprimento_m"]:
                    specs["comprimento_m"] = value
                    logger.info(f"Encontrado comprimento_m (fallback): {specs['comprimento_m']}")
                elif key == "altura_m_fallback" and not specs["altura_m"]:
                    specs["altura_m"] = value
                    logger.info(f"Encontrado altura_m (fallback): {specs['altura_m']}")
                elif key == "altura_m_cabina":
                    specs["altura_m"] = value
                    logger.info(f"Encontrado altura_m: {specs['altura_m']}")
                else:
                    specs[key] = value
                    logger.info(f"Encontrado {key}: {specs[key]}")
        else:
            if key not in ["altura_m_cabina", "altura_m_fallback", "comprimento_m_fallback"]:
                logger.info(f"Não encontrado {key} no texto")

    # Extrair motor
    for key, pattern in motor_patterns.items():
        if key in specs["motor"] and specs["motor"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["motor"][key] = value
            logger.info(f"Encontrado motor.{key}: {specs['motor'][key]}")
        else:
            logger.info(f"Não encontrado motor.{key} no texto")

    # Extrair transmissão
    for key, pattern in transmissao_patterns.items():
        if key in specs["transmissao"] and specs["transmissao"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["transmissao"][key] = value
            logger.info(f"Encontrado transmissao.{key}: {specs['transmissao'][key]}")
        else:
            logger.info(f"Não encontrado transmissao.{key} no texto")

    # Extrair embreagem
    for key, pattern in embreagem_patterns.items():
        if key in specs["embreagem"] and specs["embreagem"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["embreagem"][key] = value
            logger.info(f"Encontrado embreagem.{key}: {specs['embreagem'][key]}")
        else:
            logger.info(f"Não encontrado embreagem.{key} no texto")

    # Extrair suspensão dianteira
    for key, pattern in suspensao_dianteira_patterns.items():
        if key in specs["suspensao"]["dianteira"] and specs["suspensao"]["dianteira"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["suspensao"]["dianteira"][key] = value
            logger.info(f"Encontrado suspensao.dianteira.{key}: {specs['suspensao']['dianteira'][key]}")
        else:
            logger.info(f"Não encontrado suspensao.dianteira.{key} no texto")

    # Extrair suspensão traseira
    for key, pattern in suspensao_traseira_patterns.items():
        if key in specs["suspensao"]["traseira"] and specs["suspensao"]["traseira"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["suspensao"]["traseira"][key] = value
            logger.info(f"Encontrado suspensao.traseira.{key}: {specs['suspensao']['traseira'][key]}")
        else:
            logger.info(f"Não encontrado suspensao.traseira.{key} no texto")

    # Extrair sistema elétrico
    for key, pattern in sistema_eletrico_patterns.items():
        if key in specs["sistema_eletrico"] and specs["sistema_eletrico"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["sistema_eletrico"][key] = value
            logger.info(f"Encontrado sistema_eletrico.{key}: {specs['sistema_eletrico'][key]}")
        else:
            logger.info(f"Não encontrado sistema_eletrico.{key} no texto")

    # Extrair direção
    for key, pattern in direcao_patterns.items():
        if key in specs["direcao"] and specs["direcao"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["direcao"][key] = value
            logger.info(f"Encontrado direcao.{key}: {specs['direcao'][key]}")
        else:
            logger.info(f"Não encontrado direcao.{key} no texto")

    # Extrair pesos
    for key, pattern in pesos_patterns.items():
        if key in specs["pesos"] and specs["pesos"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = match.group(1).replace(".", "").replace(",", ".")
            specs["pesos"][key] = value
            logger.info(f"Encontrado pesos.{key}: {specs['pesos'][key]}")
        else:
            logger.info(f"Não encontrado pesos.{key} no texto")

    # Extrair chassi/dimensões
    for key, pattern in chassi_dimensoes_patterns.items():
        if key in specs["chassi_dimensoes"] and specs["chassi_dimensoes"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = match.group(1)
            if key in ["comprimento_total", "altura_longarina_solo", "altura_longarina_cabine", "largura_chassi", "largura_rodado_traseiro", "bitola_dianteira", "bitola_traseira", "balanco_dianteiro", "balanco_traseiro", "plataforma_carga", "entre_eixos"]:
                value = convert_mm_to_m(value)
            specs["chassi_dimensoes"][key] = value
            logger.info(f"Encontrado chassi_dimensoes.{key}: {specs['chassi_dimensoes'][key]}")
        else:
            logger.info(f"Não encontrado chassi_dimensoes.{key} no texto")

    # Extrair volumes de abastecimento
    for key, pattern in volumes_abastecimento_patterns.items():
        if key in specs["volumes_abastecimento"] and specs["volumes_abastecimento"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["volumes_abastecimento"][key] = value
            logger.info(f"Encontrado volumes_abastecimento.{key}: {specs['volumes_abastecimento'][key]}")
        else:
            logger.info(f"Não encontrado volumes_abastecimento.{key} no texto")

    # Extrair rodas e pneus
    for key, pattern in rodas_pneus_patterns.items():
        if key in specs["rodas_e_pneus"] and specs["rodas_e_pneus"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["rodas_e_pneus"][key] = value
            logger.info(f"Encontrado rodas_e_pneus.{key}: {specs['rodas_e_pneus'][key]}")
        else:
            logger.info(f"Não encontrado rodas_e_pneus.{key} no texto")

    # Extrair freios
    for key, pattern in freios_patterns.items():
        if key in specs["freios"] and specs["freios"][key] is not None:
            continue
        match = re.search(pattern, text, re.IGNORECASE | re.UNICODE)
        if match:
            value = clean_value(match.group(1))
            specs["freios"][key] = value
            logger.info(f"Encontrado freios.{key}: {specs['freios'][key]}")
        else:
            logger.info(f"Não encontrado freios.{key} no texto")

    return specs

# Função para processar tabelas e extrair especificações
def extract_specs_from_tables(tables):
    specs = {
        "modelo": None,
        "massa_kg": None,
        "largura_m": None,
        "altura_m": None,
        "comprimento_m": None,
        "esc": None,
        "frenagem_emergencia": None,
        "motor": {},
        "transmissao": {},
        "embreagem": {},
        "suspensao": {"dianteira": {}, "traseira": {}},
        "sistema_eletrico": {},
        "direcao": {},
        "pesos": {},
        "chassi_dimensoes": {},
        "volumes_abastecimento": {},
        "rodas_e_pneus": {},
        "freios": {}
    }

    for table in tables:
        for row in table:
            row_text = " ".join([str(cell) for cell in row if cell])
            table_specs = extract_specifications(row_text)
            for key, value in table_specs.items():
                if key in ["motor", "transmissao", "embreagem", "suspensao", "sistema_eletrico", "direcao", "pesos", "chassi_dimensoes", "volumes_abastecimento", "rodas_e_pneus", "freios"]:
                    for subkey, subvalue in value.items():
                        if subkey not in specs[key] or specs[key][subkey] is None:
                            specs[key][subkey] = subvalue
                elif value and not specs[key]:
                    specs[key] = value
    
    return specs

# Função para processar os PDFs
def process_pdfs(arquivos_ford, use_ocr=True):
    results = []

    for modelo, pdf_path in arquivos_ford.items():
        logger.info(f"Processando {modelo} ({pdf_path})...")
        
        if not os.path.exists(pdf_path):
            logger.error(f"Arquivo {pdf_path} não encontrado.")
            results.append({
                "modelo": modelo,
                "status": "Arquivo não encontrado",
                "massa_kg": None,
                "largura_m": None,
                "altura_m": None,
                "comprimento_m": None,
                "esc": None,
                "frenagem_emergencia": None,
                "motor_marca_modelo": None,
                "motor_tipo": None,
                "motor_potencia_maxima": None,
                "motor_torque_maximo": None,
                "motor_cilindrada_total": None,
                "motor_relacao_compressao": None,
                "motor_sistema_injecao": None,
                "transmissao_marca_modelo": None,
                "transmissao_acionamento": None,
                "transmissao_marchas": None,
                "transmissao_relacao_transmissao_1a": None,
                "transmissao_relacao_transmissao_2a": None,
                "transmissao_relacao_transmissao_3a": None,
                "transmissao_relacao_transmissao_4a": None,
                "transmissao_relacao_transmissao_5a": None,
                "transmissao_relacao_transmissao_re": None,
                "embreagem_marca": None,
                "embreagem_acionamento": None,
                "embreagem_tipo": None,
                "embreagem_diametro": None,
                "suspensao_dianteira_eixo": None,
                "suspensao_dianteira_molas": None,
                "suspensao_dianteira_amortecedores": None,
                "suspensao_traseira_eixo": None,
                "suspensao_traseira_molas": None,
                "suspensao_traseira_amortecedores": None,
                "sistema_eletrico_sistema": None,
                "sistema_eletrico_alternador": None,
                "sistema_eletrico_bateria": None,
                "direcao_marca_modelo": None,
                "direcao_tipo": None,
                "pesos_pbt_homologado": None,
                "pesos_carga_util": None,
                "pesos_capacidade_tracao": None,
                "chassi_dimensoes_entre_eixos": None,
                "chassi_dimensoes_balanco_dianteiro": None,
                "chassi_dimensoes_balanco_traseiro": None,
                "chassi_dimensoes_comprimento_total": None,
                "chassi_dimensoes_plataforma_carga": None,
                "chassi_dimensoes_altura_longarina_solo": None,
                "chassi_dimensoes_altura_longarina_cabine": None,
                "chassi_dimensoes_largura_chassi": None,
                "chassi_dimensoes_largura_rodado_traseiro": None,
                "chassi_dimensoes_bitola_dianteira": None,
                "chassi_dimensoes_bitola_traseira": None,
                "chassi_dimensoes_diametro_giro": None,
                "volumes_abastecimento_tanque_combustivel": None,
                "volumes_abastecimento_reservatorio_arla": None,
                "volumes_abastecimento_oleo_motor": None,
                "rodas_e_pneus_rodas": None,
                "rodas_e_pneus_medidas": None,
                "rodas_e_pneus_pneus": None,
                "freios_de_servico": None,
                "freios_de_estacionamento": None,
                "freios_freio_motor": None,
                "freios_area_efetiva": None
            })
            continue

        # Tentar extrair texto com pdfplumber
        text = extract_text_pdfplumber(pdf_path)
        
        # Tentar extrair tabelas
        tables = extract_tables_pdfplumber(pdf_path)
        
        # Se a extração de texto falhar ou retornar pouco texto, tentar OCR
        if use_ocr and (not text or len(text.strip()) < 50) and not tables:
            logger.info(f"Tentando OCR em {pdf_path}...")
            text = extract_text_ocr(pdf_path)
        
        # Salvar texto extraído em log
        save_extracted_text(modelo, text)
        
        # Verificar se o texto ou tabelas foram extraídos
        if (not text or len(text.strip()) < 50) and not tables:
            logger.error(f"Falha na extração de {pdf_path}. Verifique manualmente.")
            results.append({
                "modelo": modelo,
                "status": "Falha na extração",
                "massa_kg": None,
                "largura_m": None,
                "altura_m": None,
                "comprimento_m": None,
                "esc": None,
                "frenagem_emergencia": None,
                "motor_marca_modelo": None,
                "motor_tipo": None,
                "motor_potencia_maxima": None,
                "motor_torque_maximo": None,
                "motor_cilindrada_total": None,
                "motor_relacao_compressao": None,
                "motor_sistema_injecao": None,
                "transmissao_marca_modelo": None,
                "transmissao_acionamento": None,
                "transmissao_marchas": None,
                "transmissao_relacao_transmissao_1a": None,
                "transmissao_relacao_transmissao_2a": None,
                "transmissao_relacao_transmissao_3a": None,
                "transmissao_relacao_transmissao_4a": None,
                "transmissao_relacao_transmissao_5a": None,
                "transmissao_relacao_transmissao_re": None,
                "embreagem_marca": None,
                "embreagem_acionamento": None,
                "embreagem_tipo": None,
                "embreagem_diametro": None,
                "suspensao_dianteira_eixo": None,
                "suspensao_dianteira_molas": None,
                "suspensao_dianteira_amortecedores": None,
                "suspensao_traseira_eixo": None,
                "suspensao_traseira_molas": None,
                "suspensao_traseira_amortecedores": None,
                "sistema_eletrico_sistema": None,
                "sistema_eletrico_alternador": None,
                "sistema_eletrico_bateria": None,
                "direcao_marca_modelo": None,
                "direcao_tipo": None,
                "pesos_pbt_homologado": None,
                "pesos_carga_util": None,
                "pesos_capacidade_tracao": None,
                "chassi_dimensoes_entre_eixos": None,
                "chassi_dimensoes_balanco_dianteiro": None,
                "chassi_dimensoes_balanco_traseiro": None,
                "chassi_dimensoes_comprimento_total": None,
                "chassi_dimensoes_plataforma_carga": None,
                "chassi_dimensoes_altura_longarina_solo": None,
                "chassi_dimensoes_altura_longarina_cabine": None,
                "chassi_dimensoes_largura_chassi": None,
                "chassi_dimensoes_largura_rodado_traseiro": None,
                "chassi_dimensoes_bitola_dianteira": None,
                "chassi_dimensoes_bitola_traseira": None,
                "chassi_dimensoes_diametro_giro": None,
                "volumes_abastecimento_tanque_combustivel": None,
                "volumes_abastecimento_reservatorio_arla": None,
                "volumes_abastecimento_oleo_motor": None,
                "rodas_e_pneus_rodas": None,
                "rodas_e_pneus_medidas": None,
                "rodas_e_pneus_pneus": None,
                "freios_de_servico": None,
                "freios_de_estacionamento": None,
                "freios_freio_motor": None,
                "freios_area_efetiva": None
            })
            continue
        
        # Extrair especificações do texto
        specs = extract_specifications(text)
        
        # Se houver tabelas, extrair especificações delas também
        if tables:
            table_specs = extract_specs_from_tables(tables)
            for key, value in table_specs.items():
                if key in ["motor", "transmissao", "embreagem", "suspensao", "sistema_eletrico", "direcao", "pesos", "chassi_dimensoes", "volumes_abastecimento", "rodas_e_pneus", "freios"]:
                    for subkey, subvalue in value.items():
                        if subkey not in specs[key] or specs[key][subkey] is None:
                            specs[key][subkey] = subvalue
                elif value and not specs[key]:
                    specs[key] = value
        
        # Estruturar o resultado para o Excel
        result = {
            "modelo": modelo,
            "status": "Sucesso",
            "massa_kg": specs["massa_kg"],
            "largura_m": specs["largura_m"],
            "altura_m": specs["altura_m"],
            "comprimento_m": specs["comprimento_m"],
            "esc": specs["esc"],
            "frenagem_emergencia": specs["frenagem_emergencia"],
            "motor_marca_modelo": specs["motor"].get("marca_modelo"),
            "motor_tipo": specs["motor"].get("tipo"),
            "motor_potencia_maxima": specs["motor"].get("potencia_maxima"),
            "motor_torque_maximo": specs["motor"].get("torque_maximo"),
            "motor_cilindrada_total": specs["motor"].get("cilindrada_total"),
            "motor_relacao_compressao": specs["motor"].get("relacao_compressao"),
            "motor_sistema_injecao": specs["motor"].get("sistema_injecao"),
            "transmissao_marca_modelo": specs["transmissao"].get("marca_modelo"),
            "transmissao_acionamento": specs["transmissao"].get("acionamento"),
            "transmissao_marchas": specs["transmissao"].get("marchas"),
            "transmissao_relacao_transmissao_1a": specs["transmissao"].get("relacao_transmissao_1a"),
            "transmissao_relacao_transmissao_2a": specs["transmissao"].get("relacao_transmissao_2a"),
            "transmissao_relacao_transmissao_3a": specs["transmissao"].get("relacao_transmissao_3a"),
            "transmissao_relacao_transmissao_4a": specs["transmissao"].get("relacao_transmissao_4a"),
            "transmissao_relacao_transmissao_5a": specs["transmissao"].get("relacao_transmissao_5a"),
            "transmissao_relacao_transmissao_re": specs["transmissao"].get("relacao_transmissao_re"),
            "embreagem_marca": specs["embreagem"].get("marca"),
            "embreagem_acionamento": specs["embreagem"].get("acionamento"),
            "embreagem_tipo": specs["embreagem"].get("tipo"),
            "embreagem_diametro": specs["embreagem"].get("diametro"),
            "suspensao_dianteira_eixo": specs["suspensao"]["dianteira"].get("eixo"),
            "suspensao_dianteira_molas": specs["suspensao"]["dianteira"].get("molas"),
            "suspensao_dianteira_amortecedores": specs["suspensao"]["dianteira"].get("amortecedores"),
            "suspensao_traseira_eixo": specs["suspensao"]["traseira"].get("eixo"),
            "suspensao_traseira_molas": specs["suspensao"]["traseira"].get("molas"),
            "suspensao_traseira_amortecedores": specs["suspensao"]["traseira"].get("amortecedores"),
            "sistema_eletrico_sistema": specs["sistema_eletrico"].get("sistema"),
            "sistema_eletrico_alternador": specs["sistema_eletrico"].get("alternador"),
            "sistema_eletrico_bateria": specs["sistema_eletrico"].get("bateria"),
            "direcao_marca_modelo": specs["direcao"].get("marca_modelo"),
            "direcao_tipo": specs["direcao"].get("tipo"),
            "pesos_pbt_homologado": specs["pesos"].get("pbt_homologado"),
            "pesos_carga_util": specs["pesos"].get("carga_util"),
            "pesos_capacidade_tracao": specs["pesos"].get("capacidade_tracao"),
            "chassi_dimensoes_entre_eixos": specs["chassi_dimensoes"].get("entre_eixos"),
            "chassi_dimensoes_balanco_dianteiro": specs["chassi_dimensoes"].get("balanco_dianteiro"),
            "chassi_dimensoes_balanco_traseiro": specs["chassi_dimensoes"].get("balanco_traseiro"),
            "chassi_dimensoes_comprimento_total": specs["chassi_dimensoes"].get("comprimento_total"),
            "chassi_dimensoes_plataforma_carga": specs["chassi_dimensoes"].get("plataforma_carga"),
            "chassi_dimensoes_altura_longarina_solo": specs["chassi_dimensoes"].get("altura_longarina_solo"),
            "chassi_dimensoes_altura_longarina_cabine": specs["chassi_dimensoes"].get("altura_longarina_cabine"),
            "chassi_dimensoes_largura_chassi": specs["chassi_dimensoes"].get("largura_chassi"),
            "chassi_dimensoes_largura_rodado_traseiro": specs["chassi_dimensoes"].get("largura_rodado_traseiro"),
            "chassi_dimensoes_bitola_dianteira": specs["chassi_dimensoes"].get("bitola_dianteira"),
            "chassi_dimensoes_bitola_traseira": specs["chassi_dimensoes"].get("bitola_traseira"),
            "chassi_dimensoes_diametro_giro": specs["chassi_dimensoes"].get("diametro_giro"),
            "volumes_abastecimento_tanque_combustivel": specs["volumes_abastecimento"].get("tanque_combustivel"),
            "volumes_abastecimento_reservatorio_arla": specs["volumes_abastecimento"].get("reservatorio_arla"),
            "volumes_abastecimento_oleo_motor": specs["volumes_abastecimento"].get("oleo_motor"),
            "rodas_e_pneus_rodas": specs["rodas_e_pneus"].get("rodas"),
            "rodas_e_pneus_medidas": specs["rodas_e_pneus"].get("medidas"),
            "rodas_e_pneus_pneus": specs["rodas_e_pneus"].get("pneus"),
            "freios_de_servico": specs["freios"].get("de_servico"),
            "freios_de_estacionamento": specs["freios"].get("de_estacionamento"),
            "freios_freio_motor": specs["freios"].get("freio_motor"),
            "freios_area_efetiva": specs["freios"].get("area_efetiva")
        }
        
        results.append(result)
    
    return results

# Função para salvar resultados em Excel
def save_to_excel(results, output_file):
    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False, engine='openpyxl')
    logger.info(f"Resultados salvos em {output_file}")

# Executar o processamento
results = process_pdfs(arquivos_ford, use_ocr=True)
save_to_excel(results, saida)

SCANIA PDF

In [ ]:
import pdfplumber
import pandas as pd
import re

pdf_paths = {
    "Scania - Longas Distancias.pdf": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Scania\Scania - Longas Distancias.pdf",
    "Scania - Mineração.pdf": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Scania\Scania - Mineração.pdf",
    "Scania - Cana - Madeira.pdf": r"C:\Users\gabriel.vinicius\Documents\Vscode\Web Scraping\Scania\Scania - Cana - Madeira.pdf"
}

cabines, motores_tabela, caixas, diferenciais, eixos = [], [], [], [], []
tecnologias_set = set()

# Cabines manuais
cabines_manualmente = {
    "Scania - Longas Distancias.pdf": [
        ("S", "CS20H - Alta", "Cabine premium para viagens de longa distância", "560 a 770 hp"),
        ("R", "CR20N - Normal / CR20H - Alta", "Conforto com múltiplas configurações", "370 a 660 hp"),
        ("G", "CG20N - Normal / CG20H - Alta", "Robustez com versatilidade", "320 a 500 hp"),
        ("P", "CP20N - Normal", "Leve, prática e eficiente", "280 a 450 hp")
    ],
    "Scania - Mineração.pdf": [
        ("R", "CR17N - Normal", "Alta resistência para terreno severo", "370 a 660 hp"),
        ("R", "CR20N - Normal", "Cabine reforçada para operação intensa", "410 a 770 hp"),
        ("G", "CG14L - Baixa", "Baixo centro de gravidade para estabilidade", "370 a 500 hp"),
        ("P", "CP14L - Baixa", "Compacta e funcional", "320 a 410 hp")
    ],
    "Scania - Cana - Madeira.pdf": [
        ("R", "CR17N - Normal", "Equilíbrio entre força e conforto", "370 a 660 hp"),
        ("R", "CR20N - Normal", "Para operação contínua e pesada", "410 a 770 hp"),
        ("G", "CG14L - Baixa", "Chassi baixo ideal para campo", "360 a 500 hp"),
        ("P", "CP14L - Baixa", "Compacta e leve", "320 a 410 hp")
    ]
}

pagina_motores_por_pdf = {
    "Scania - Longas Distancias.pdf": None,
    "Scania - Mineração.pdf": 18,
    "Scania - Cana - Madeira.pdf": 14
}

motores_manualmente = {
    "Scania - Longas Distancias.pdf": [
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "280 hp (206 kW) a 1.900 rpm", "1.400 Nm entre 1.000–1.400 rpm", "Scania SCR"),
        ("5 Cilindros", "9 litros", "GNC/Biometano", "280 hp (206 kW) a 1.900 rpm", "1.350 Nm entre 1.000–1.400 rpm", "Scania EGR"),
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "320 hp (235 kW) a 1.900 rpm", "1.600 Nm entre 1.050–1.400 rpm", "Scania SCR"),
        ("5 Cilindros", "9 litros", "GNC/Biometano", "340 hp (250 kW) a 1.900 rpm", "1.600 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "360 hp (265 kW) a 1.900 rpm", "1.700 Nm entre 1.050–1.475 rpm", "Scania SCR"),
        ("6 Cilindros", "13 litros", "GNC/Biometano", "410 hp (302 kW) a 1.900 rpm", "2.000 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "420 hp (309 kW) a 1.800 rpm", "2.300 Nm entre 900–1.280 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "460 hp (338 kW) a 1.800 rpm", "2.500 Nm entre 900–1.290 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "500 hp (368 kW) a 1.800 rpm", "2.550 Nm entre 925–1.340 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "560 hp (412 kW) a 1.800 rpm", "2.800 Nm entre 900–1.400 rpm", "Scania Twin SCR"),
        ("8 Cilindros em V", "16 litros", "Diesel/Biodiesel", "660 hp (485 kW) a 1.900 rpm", "3.300 Nm entre 950–1.400 rpm", "Scania Twin SCR"),
        ("8 Cilindros em V", "16 litros", "Diesel/Biodiesel", "770 hp (566 kW) a 1.800 rpm", "3.700 Nm entre 1.000–1.450 rpm", "Scania Twin SCR")
    ],
    "Scania - Mineração.pdf": [
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "320 hp (235 kW) a 1.900 rpm", "1.600 Nm entre 1.050–1.400 rpm", "Scania SCR"),
        ("5 Cilindros", "9 litros", "GNC/Biometano", "340 hp (250 kW) a 1.900 rpm", "1.600 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "360 hp (265 kW) a 1.900 rpm", "1.700 Nm entre 1.050–1.350 rpm", "Scania SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "370 hp (272 kW) a 1.800 rpm", "1.900 Nm entre 900–1.340 rpm", "Scania SCR"),
        ("6 Cilindros", "13 litros", "GNC/Biometano", "410 hp (302 kW) a 1.900 rpm", "2.000 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "420 hp (309 kW) a 1.800 rpm", "2.300 Nm entre 900–1.280 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "460 hp (338 kW) a 1.800 rpm", "2.500 Nm entre 900–1.290 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "500 hp (368 kW) a 1.800 rpm", "2.650 Nm entre 900–1.320 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "560 hp (412 kW) a 1.800 rpm", "2.800 Nm entre 900–1.400 rpm", "Scania Twin SCR"),
        ("8 Cilindros em V", "16 litros", "Diesel/Biodiesel", "660 hp (485 kW) a 1.900 rpm", "3.300 Nm entre 950–1.400 rpm", "Scania Twin SCR"),
        ("8 Cilindros em V", "16 litros", "Diesel/Biodiesel", "770 hp (566 kW) a 1.800 rpm", "3.700 Nm entre 1.000–1.450 rpm", "Scania Twin SCR")
    ],
    "Scania - Cana - Madeira.pdf": [
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "320 hp (235 kW) a 1.900 rpm", "1.600 Nm entre 1.050–1.400 rpm", "Scania SCR"),
        ("5 Cilindros", "9 litros", "GNC/Biometano", "340 hp (250 kW) a 1.900 rpm", "1.600 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("5 Cilindros", "9 litros", "Diesel/Biodiesel", "360 hp (265 kW) a 1.900 rpm", "1.700 Nm entre 1.050–1.350 rpm", "Scania SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "410 hp (302 kW) a 1.900 rpm", "2.000 Nm entre 1.100–1.400 rpm", "Scania EGR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "420 hp (309 kW) a 1.800 rpm", "2.300 Nm entre 900–1.280 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "460 hp (338 kW) a 1.800 rpm", "2.500 Nm entre 900–1.290 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "500 hp (368 kW) a 1.800 rpm", "2.650 Nm entre 900–1.320 rpm", "Scania Twin SCR"),
        ("6 Cilindros", "13 litros", "Diesel/Biodiesel", "560 hp (412 kW) a 1.800 rpm", "2.800 Nm entre 900–1.400 rpm", "Scania Twin SCR"),
        ("8 Cilindros em V", "16 litros", "Diesel/Biodiesel", "660 hp (485 kW) a 1.900 rpm", "3.300 Nm entre 950–1.400 rpm", "Scania Twin SCR")
    ]
}

# Processamento
for filename, pdf_path in pdf_paths.items():
    with pdfplumber.open(pdf_path) as pdf:
        for serie, modelo, desc, motor in cabines_manualmente[filename]:
            cabines.append({"PDF": filename, "Série": serie, "Modelos": modelo, "Descrição": desc, "Motores": motor})

        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if not text:
                continue

            for termo in ["ADAS", "AEB", "ACC", "LDW"]:
                if termo in text:
                    tecnologias_set.add(termo)

            if re.search(r"R\d{3}", text):
                matches = re.findall(r"(R[\w\+]+)\s+(\d{2,3}\.\d{3}) kg\s+([\d\s\/\.,]+)", text)
                for modelo, cmt, relacoes in matches:
                    diferenciais.append({"PDF": filename, "Modelo": modelo.strip(), "CMT": cmt.strip(), "Relações de Diferenciais": relacoes.strip()})

            if "CAVALO MECÂNICO" in text and "RÍGIDO" in text:
                for linha in text.strip().split("\n"):
                    if "CAVALO MECÂNICO" in linha:
                        secao = "Cavalo Mecânico"
                    elif "RÍGIDO" in linha:
                        secao = "Rígido"
                    elif re.match(r"^\d+x\d", linha.strip()):
                        partes = linha.strip().split()
                        eixo = partes[0]
                        susp = " ".join(partes[1:]) if len(partes) > 1 else ""
                        eixos.append({"PDF": filename, "Aplicação": secao, "Configuração": eixo, "Suspensão": susp})

        # Motores manuais
        if filename in motores_manualmente:
            for bloco, cilindrada, combustivel, potencia, torque, emissao in motores_manualmente[filename]:
                motores_tabela.append({
                    "PDF": filename,
                    "Bloco": bloco,
                    "Cilindrada": cilindrada,
                    "Combustível": combustivel,
                    "Potência": potencia,
                    "Torque": torque,
                    "Emissão": emissao
                })

        # Caixas de Câmbio
        if filename == "Scania - Cana - Madeira.pdf":
            caixas.extend([
                {"PDF": filename, "Caixa": "G25CM", "Motores": "420 a 460 hp", "Velocidades": "12 velocidades", "Especificação": "1 marcha super reduzida e 1 overdrive"},
                {"PDF": filename, "Caixa": "G33CM", "Motores": "500 a 660 hp", "Velocidades": "12 velocidades", "Especificação": "1 marcha super reduzida e 1 overdrive"},
                {"PDF": filename, "Caixa": "GRS 905", "Motores": "500 a 560 hp", "Velocidades": "14 velocidades", "Especificação": "2 marchas superlentas e 1 marcha directdrive"},
                {"PDF": filename, "Caixa": "GRSO 905R", "Motores": "500 a 560 hp", "Velocidades": "14 velocidades", "Especificação": "2 marchas superlentas e 1 marcha overdrive"},
                {"PDF": filename, "Caixa": "GRSO 935R", "Motores": "660 a 770 hp", "Velocidades": "14 velocidades", "Especificação": "2 marchas superlentas e 1 marcha overdrive"}
            ])
        else:
            for page in pdf.pages:
                text = page.extract_text()
                if not text:
                    continue
                matches = re.findall(
                    r"(G\d{2}CM|G\d{2}|GRS905(?:/R)?|GRSO9\d{2}(?:/R)?)\s+([^\n]+?)\s+(\d{1,2} velocidades)\s+(.+?)(?:\n|$)",
                    text)
                for modelo, motores_txt, velocidades, detalhes in matches:
                    caixas.append({
                        "PDF": filename,
                        "Caixa": modelo.strip(),
                        "Motores": motores_txt.strip(),
                        "Velocidades": velocidades.strip(),
                        "Especificação": detalhes.strip()
                    })

# Exporta para Excel
with pd.ExcelWriter(r"C:\\Users\\gabriel.vinicius\\Documents\\Vscode\\Web Scraping\\Scania Base.xlsx") as writer:
    pd.DataFrame(cabines).to_excel(writer, sheet_name="Cabines", index=False)
    pd.DataFrame(motores_tabela).to_excel(writer, sheet_name="Motores Proconve P8", index=False)
    pd.DataFrame(caixas).to_excel(writer, sheet_name="Caixas de Câmbio", index=False)
    pd.DataFrame(diferenciais).to_excel(writer, sheet_name="Diferenciais", index=False)
    pd.DataFrame(eixos).to_excel(writer, sheet_name="Eixos", index=False)
    pd.DataFrame(sorted(tecnologias_set), columns=["Tecnologias"]).to_excel(writer, sheet_name="Tecnologias", index=False)

print("Extração completa com dados manuais e automáticos consolidada.")

In [3]:
import pandas as pd

# Modelo
modelo = "C-816"

# Dicionário com os dados organizados
dados = {
    "Modelo": modelo,

    # MOTOR
    "Motor - Marca / Modelo": "Cummins ISB4.5 160 P7-1",
    "Motor - Tipo": "Diesel - 4 cilindros em linha",
    "Motor - Potência máxima (cv/kW) / rpm": "162cv (119kW) a 2.300 rpm",
    "Motor - Torque máximo (kgfm/Nm) / rpm": "56,1 kgfm (550 Nm) / 1.100-2.000 rpm",
    "Motor - Cilindrada total (cm3)": "4.462",
    "Motor - Relação de compressão": "17,3 : 1",
    "Motor - Sistema de injeção": "Injeção Eletrônica - Common Rail",
    "Motor - Norma de ensaio": "NBR ISO 1585",

    # TRANSMISSÃO
    "Transmissão - Marca/Modelo": "Eaton/ FSO-4505 C",
    "Transmissão - Acionamento": "Manual mecânico",
    "Transmissão - Marchas": "5 marchas à frente e 1 ré",
    "Transmissão - Relação 1ª marcha": "5,78:1",
    "Transmissão - Relação 2ª marcha": "2,73:1",
    "Transmissão - Relação 3ª marcha": "1,63:1",
    "Transmissão - Relação 4ª marcha": "1:1",
    "Transmissão - Relação 5ª marcha": "0,77:1",
    "Transmissão - Relação ré": "5,26:1",

    # EMBREAGEM
    "Embreagem - Marca": "Sachs",
    "Embreagem - Acionamento": "Hidráulico",
    "Embreagem - Tipo": "Disco simples orgânico a seco, com platô acionado por mola diafragma",
    "Embreagem - Diâmetro (mm)": "330",

    # EIXO TRASEIRO MOTRIZ
    "Eixo Traseiro - Marca / Modelo": "Dana/ 480",
    "Eixo Traseiro - Redução": "3,90:1",

    # SUSPENSÃO
    "Suspensão Dianteira - Eixo": "Eixo rígido em aço forjado, com barra estabilizadora",
    "Suspensão Dianteira - Molas": "Feixe de molas parabólicas",
    "Suspensão Dianteira - Amortecedores": "Telescópicos, hidráulicos de dupla ação",
    "Suspensão Traseira - Eixo": "Eixo rígido em aço estampado com barra estabilizadora",
    "Suspensão Traseira - Molas": "Feixe de molas semielípticas (principal) e parabólicas (auxiliar)",
    "Suspensão Traseira - Amortecedores": "Telescópicos, hidráulicos de dupla ação",

    # SISTEMA ELÉTRICO
    "Elétrico - Sistema": "12V",
    "Elétrico - Alternador": "14 / 90",
    "Elétrico - Bateria": "1 / 12 / 750 / 100",

    # RODAS E PNEUS
    "Rodas e Pneus - Rodas": "Aço estampado",
    "Rodas e Pneus - Medidas": "17,5 x 6",
    "Rodas e Pneus - Pneus": "215 / 75R17,5”",

    # FREIOS
    "Freios - Serviço": "A ar, tipo S Cam com circuito duplo, dianteiros e traseiros a tambor",
    "Freios - ABS/ASR/EBD": "",
    "Freios - Estacionamento": "A ar com molas acumuladoras e atuação nas rodas traseiras",
    "Freios - Freio motor": "Válvula tipo borboleta no tubo do escapamento, com acionamento eletropneumático",
    "Freios - Área efetiva de frenagem (cm²)": "2160",

    # DIREÇÃO
    "Direção - Marca/Modelo": "ZF 8090",
    "Direção - Tipo": "Hidráulica, com esferas recirculantes",

    # DESEMPENHO
    "Desempenho - Relação de redução do eixo traseiro": "3,90:1",
    "Desempenho - Velocidade máxima (km/h)": "108 / 108",
    "Desempenho - Capacidade máxima de subida (%)": "38 / 27",
    "Desempenho - Partida em rampa (%)": "29 / 21",

    # PESOS
    "Pesos - Entre eixos (mm)": "3.300 / 3.900 / 4.300",
    "Pesos - Ordem de marcha (dianteiro)": "2.140 / 2.165 / 2.180",
    "Pesos - Ordem de marcha (traseiro)": "1.000 / 1.005 / 1.010",
    "Pesos - Ordem de marcha (total)": "3.140 / 3.170 / 3.190",
    "Pesos - Limite eixo dianteiro": "3.000",
    "Pesos - Limite eixo traseiro": "5.250",
    "Pesos - Limite total admissível": "8.250",
    "Pesos - PBT Homologado": "8.250",
    "Pesos - Carga útil + carroceria": "5.110 / 5.080 / 5.060",
    "Pesos - CMT": "11.000",

    # LONGARINA
    "Longarina - Material": "LNE 600",

    # VOLUMES DE ABASTECIMENTO
    "Abastecimento - Tanque combustível (L)": "150",
    "Abastecimento - Arla 32 (L)": "25",
    "Abastecimento - Óleo motor com filtro (L)": "13",

    # CHASSI/DIMENSÕES
    "Dimensões - Entre eixos": "3.300 / 3.900 / 4.300",
    "Dimensões - Balanço dianteiro": "1.270",
    "Dimensões - Balanço traseiro": "1.570 / 1.840 / 1.840",
    "Dimensões - Comprimento total": "6.140 / 7.010 / 7.410",
    "Dimensões - Distância carroceria ao eixo dianteiro": "650",
    "Dimensões - Plataforma de carga": "4.300 / 5.090 / 5.490",
    "Dimensões - Altura da longarina ao solo": "690",
    "Dimensões - Altura da longarina à cabine": "1.780",
    "Dimensões - Altura máxima da cabine basculada": "3.000",
    "Dimensões - Largura externa do chassi": "865",
    "Dimensões - Largura máxima rodado traseiro": "2.198",
    "Dimensões - Bitola dianteira": "1.880",
    "Dimensões - Bitola traseira": "1.670",
    "Dimensões - Diâmetro de giro (m)": "13,40 / 15,60 / 17,80",
}

# Criar DataFrame
df = pd.DataFrame([dados])

# Salvar no Excel
df.to_excel("Ficha_Tecnica_C816.xlsx", index=False)

print("Ficha técnica do modelo C-816 salva com sucesso!")


Ficha técnica do modelo C-816 salva com sucesso!
